In [1]:
import netgen.gui
from ngsolve import *
import numpy as np


from ngsolve.solvers import CG

In [2]:
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.5))
Draw (mesh)

In [3]:
fes = H1(mesh, order=3,dirichlet=".*")
gfu_a = GridFunction(fes,name = "u_a")
gfu_b = GridFunction(fes,name = "u_b")

In [4]:
ua = fes.TrialFunction()
va = fes.TestFunction()

Aa = BilinearForm(fes)
Fa = LinearForm(fes)

Aa += ua*va*dx

Fa += va*dx

Aa.Assemble()
Fa.Assemble()

c = Preconditioner(Aa,"local",test = True)
c.Update()

#gfu_a.vec.data = Aa.mat.Inverse(fes.FreeDofs()) * Fa.vec

with TaskManager():
    CG (mat=Aa.mat, pre=c.mat, rhs=Fa.vec, sol=gfu_a.vec,printrates=True, maxsteps=10000,tol=1e-9)
Draw(gfu_a)

#Draw(gfu_a)

iteration 0 error = 1.3964240043768876
iteration 1 error = 0.2490974131598827
iteration 2 error = 0.06059874043906732
iteration 3 error = 0.07504739965452975
iteration 4 error = 0.022041316810647506
iteration 5 error = 1.3901873556413582e-15


In [5]:
ub = fes.TrialFunction()
vb = fes.TestFunction()

Ab = BilinearForm(fes)
Fb = LinearForm(fes)

Ab += grad(ub)*grad(vb)*dx

Fb += vb*dx

Ab.Assemble()
Fb.Assemble()

gfu_b.vec.data = Ab.mat.Inverse(fes.FreeDofs()) * Fb.vec

Draw(gfu_b)

In [1]:
j = 0
num_a = []
d_a = []
for hi in range(1,20):
    h = 0.5/hi
    print("num: ",j,"maxh",h)
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))
    fes = H1(mesh, order=6,dirichlet=".*")
    gfu_a = GridFunction(fes,name = "u_a")
    ua = fes.TrialFunction()
    va = fes.TestFunction()

    Aa = BilinearForm(fes)
    Fa = LinearForm(fes)

    Aa += ua*va*dx

    Fa += va*dx

    Aa.Assemble()
    Fa.Assemble()

    c = Preconditioner(Aa,"local",test = True)
    c.Update()
    data = []
    callback = lambda k,r: data.append((k,r))
    #gfu_a.vec.data = Aa.mat.Inverse(fes.FreeDofs()) * Fa.vec

    with TaskManager():
        CG (mat=Aa.mat, pre=c.mat, rhs=Fa.vec, sol=gfu_a.vec,printrates=False, maxsteps=10000,tol=1e-15,callback = callback)
    
    num_a.append(len(data))
    d_a.append(h)
    j = j + 1
Draw (mesh)
Draw(gfu_a)

num:  0 maxh 0.5


NameError: name 'Mesh' is not defined

In [ ]:
import matplotlib.pyplot as plt

plt.plot(d,num)

In [ ]:
j = 0
num_b = []
d_b = []
for hi in range(1,20):
    h = 0.5/hi
    print("num: ",j,"maxh",h)
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))
    fes = H1(mesh, order=6,dirichlet=".*")
    gfu_b = GridFunction(fes,name = "u_b")
    ub = fes.TrialFunction()
    vb = fes.TestFunction()

    Ab = BilinearForm(fes)
    Fb = LinearForm(fes)

    Ab += grad(ub)*grad(vb)*dx

    Fb += vb*dx

    Ab.Assemble()
    Fb.Assemble()

    c = Preconditioner(Ab,"local",test = True)
    c.Update()
    data = []
    callback = lambda k,r: data.append((k,r))
    #gfu_a.vec.data = Aa.mat.Inverse(fes.FreeDofs()) * Fa.vec

    with TaskManager():
        CG (mat=Ab.mat, pre=c.mat, rhs=Fb.vec, sol=gfu_b.vec,printrates=False, maxsteps=10000,tol=1e-17,callback = callback)
    
    num_b.append(len(data))
    d_b.append(h)
    j = j + 1
Draw (mesh)
Draw(gfu_b)

In [ ]:
plt.plot(d_a,num_a)
plt.plot(d_b,num_b)

In [ ]:
ref_b = []
for i in range(len(num_b)):
    ref_b.append(100+7.5/d_b[i])

In [ ]:
ref_b

In [ ]:
import matplotlib.pyplot as plt
plt.title("kondNum vs h")
plt.grid()
plt.xlabel("h")
plt.ylabel("number of iterations")
plt.plot(d_a,num_a,label='kondNum of the massmatrix')
plt.plot(d_b,num_b,label='kondNum of the stiffnesmatrix')
plt.plot(d_b,ref_b,label='estimate c+O(1/h)')
plt.legend()